<a href="https://colab.research.google.com/github/abrahamjenie/Programming-for-Data-Analysis-CA1-Web-Scraping/blob/main/SalaryData_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Importing Packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import lxml
import time

In [7]:
url="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin"
r = requests.get(url)
print(r) #status code is 200 

<Response [200]>


In [8]:
r.content
1

1

In [9]:
soup=BeautifulSoup(r.content,'html.parser')
script=soup.find_all('script')[5].text
script

'\r\n    {\r\n    "@context": "http://schema.org",\r\n    "@type": "Occupation",\r\n    "name": "Data Scientist/Analytics , Intermediate",\r\n    "mainEntityOfPage": {\r\n    "@type": "WebPage",\r\n    "lastReviewed": "2021-03-19T00:00:00Z"\r\n    },\r\n    "description": "A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.\r\nHowever, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.\r\nDay-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.\r\n",\r\n    "estimatedSalary": [{\r\n    "@type": "MonetaryAmountDistribution",\r\n    "name": "base",\r\n    "currency": "EUR",\r\n    "duration": "P1Y",\r\n    "percentile25": "31559",\r\n    "median": "39009",\r\n    "percentile75": "46472"\r\n    }]\r\n        \r\n            ,\r\n            "occupationLocation": [{\r\n            "@type": "City

In [10]:
json_data=json.loads(script,strict=False) #strict=False is used because there are control characters like '\r','\n'
json_data

{'@context': 'http://schema.org',
 '@type': 'Occupation',
 'name': 'Data Scientist/Analytics , Intermediate',
 'mainEntityOfPage': {'@type': 'WebPage',
  'lastReviewed': '2021-03-19T00:00:00Z'},
 'description': 'A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.\r\nHowever, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.\r\nDay-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.\r\n',
 'estimatedSalary': [{'@type': 'MonetaryAmountDistribution',
   'name': 'base',
   'currency': 'EUR',
   'duration': 'P1Y',
   'percentile25': '31559',
   'median': '39009',
   'percentile75': '46472'}],
 'occupationLocation': [{'@type': 'City', 'name': 'Dublin, Ireland'}]}

In [11]:
meta_url=soup.find('meta',property="og:url")
meta_url

<meta content="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin" property="og:url"/>

In [12]:
job_designation=json_data['name']
job_desc=json_data['description']
percentile25_salary=json_data['estimatedSalary'][0]['percentile25']
median_salary=json_data['estimatedSalary'][0]['median']
percentile75_salary=json_data['estimatedSalary'][0]['percentile75']
salary_type=json_data['estimatedSalary'][0]['name']
currency=json_data['estimatedSalary'][0]['currency']
salary_duration=json_data['estimatedSalary'][0]['duration']
location=json_data['occupationLocation'][0]['name']
last_reviewed_date=json_data['mainEntityOfPage']['lastReviewed']
weblink=meta_url['content']

print(job_designation,percentile25_salary,median_salary,percentile75_salary,salary_type,currency,salary_duration,location,last_reviewed_date,meta_url,weblink)

Data Scientist/Analytics , Intermediate 31559 39009 46472 base EUR P1Y Dublin, Ireland 2021-03-19T00:00:00Z <meta content="https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin" property="og:url"/> https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin


In [13]:
cities=['dublin','cork','galway','limerick','shannon-west','waterford']
job_roles=['entry-financial-analyst','entry-accountant','graphic-designer-ii','healthcare-services-planning-director','instructional-designer']
data={}
salary_df=pd.DataFrame(data)
for role in job_roles:
  for city in cities:
    template = 'https://www.salary.com/research/ie-salary/alternate/{}-salary/ie/{}'
    url = template.format(role, city)
    r = requests.get(url)
    
    soup=BeautifulSoup(r.content,'html.parser')
    script=soup.find_all('script')[5].text
    
    json_data=json.loads(script,strict=False)
    
    meta_url=soup.find('meta',property="og:url")
    
    new_row={'job_designation':json_data['name'],'percentile25_salary':json_data['estimatedSalary'][0]['percentile25'],
             'median_salary':json_data['estimatedSalary'][0]['median'],'percentile75_salary':json_data['estimatedSalary'][0]['percentile75'],
             'salary_type':json_data['estimatedSalary'][0]['name'],'currency':json_data['estimatedSalary'][0]['currency'],
             'salary_duration':json_data['estimatedSalary'][0]['duration'],'location':json_data['occupationLocation'][0]['name'],
             'last_reviewed_date':json_data['mainEntityOfPage']['lastReviewed'],'weblink':meta_url['content'],'job_desc':json_data['description']}
    salary_df = salary_df.append(new_row, ignore_index=True)
print(salary_df)

                          job_designation percentile25_salary median_salary  \
0                 Entry Financial Analyst               35028         40101   
1                 Entry Financial Analyst               33205         38013   
2                 Entry Financial Analyst               32426         37122   
3                 Entry Financial Analyst               32746         37488   
4                 Entry Financial Analyst               32103         36752   
5                 Entry Financial Analyst               31503         36066   
6                        Entry Accountant               30577         36114   
7                        Entry Accountant               28986         34234   
8                        Entry Accountant               28306         33431   
9                        Entry Accountant               28585         33761   
10                       Entry Accountant               28024         33098   
11                       Entry Accountant           